In [ ]:
import tensorflow as tf

print(tf.__version__)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
# import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

# import wandb
# from wandb.keras import WandbCallback
# wandb.init(project="teest")

import datetime

import pickle

%matplotlib inline
%config InlineBackend.figure_format='retina'
# sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 50, 250
rcParams['xtick.labelsize']=25
rcParams['ytick.labelsize']=25
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import numpy as np

from os import walk

import os

# os.environ["CUDA_DEVICE_ORDER"] = "GPU"  # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17588821609882859490
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3818241757328156010
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8144745730163242276
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7737305351368038447
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3362127872
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 240832388379309206
physical_device_desc: "device: 0, name: Quadro P2000, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i : (i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# # vytvorenie pd
# path_to_data = "/home/kriza/programing/clanok/data/unziped"

# f = []
# for (dirpath, dirnames, filenames) in walk(path_to_data):
#     f.extend(filenames)
#     break
# f.sort()

# f10m = f[:int(len(f)/2)]
# f30m = f[int(len(f)/2):]
# df10 = pd.read_csv(path_to_data + "/" + f30m[0], header=None)
# df10.index = [f30m[0][28:-15]]

# for i in f30m[1:]:
#     item = pd.read_csv(path_to_data + "/" + i, header=None)
#     item.index = [i[28:-15]]
#     df10 = pd.concat([df10, item])

# df10 = df10.T

# df10 = df10.replace("None", -1)
# df10 = df10.fillna( -1)
# df10 = df10.astype('int64')

# pickle.dump( df10, open( "/home/kriza/programing/clanok/data/picklnute/30min.p", "wb" ) )


In [2]:
df = pickle.load( open( "/home/kriza/programing/clanok/data/picklnute/30min.p", "rb" ) )


In [21]:
# hladanie pozicii bez dat
ranges_minus_1 = []
for col in df.columns: 
    ranges = []
    start = False
    begin = 0
    for i in range(df.shape[0]):
        if df[col][i] == -1:
            if not start:
                start = True
                begin = i
                continue
        if start and df[col][i] != -1:
            ranges.append((begin, i))
            start = False
    if start:
        ranges.append((begin, df.shape[0] - 1))
    ranges_minus_1.append(ranges)

In [22]:
ranges_minus_1

[[(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)],
 [(15552, 15600), (17464, 17472)]]

In [ ]:
train_range = range(0, 46656)
valid_range = range(46800 + 1, 52390)

In [ ]:
# slovnik = {
#     "56": "Counter po minutach",
#     "57": "Counter IP-ciek po minutach",
#     "58": "Counter po minutach (iny sposob ako 56)",
#     "59": "ITEM_Recon_Scanning",
#     "60": "ITEM_AVAILABILITY_DDOS",
#     "61": "ITEM_ATTEMPT_LOGIN",
#     "62": "ITEM_ATTEMPT_EXPLOIT",
#     "63": "ITEM_MALWARE_RANSOMWARE",
#     "64": "ITEM_INTRUSION_BOTNET",
#     "65": "ITEM_PORT_21",
#     "66": "ITEM_PORT_22",
#     "67": "ITEM_PORT_23",
#     "68": "ITEM_PORT_25",
#     "69": "ITEM_PORT_80",
#     "70": "ITEM_PORT_443",
#     "71": "ITEM_PORT_445",
#     "72": "ITEM_PROTO_TCP",
#     "73": "ITEM_PROTO_SSH",
#     "74": "ITEM_PROTO_UDP",
#     "75": "ITEM_PROTO_ICMP",
#     "76": "ITEM_PROTO_MSWBTSERVER",
#     "77": "ITEM_PROTO_TELNET"
# }
# # kreslenie vsetkych grafov
# fig, axs = plt.subplots(len(df.columns))
# for i in range(len(df.columns)):
#     axs[i].plot(df[df.columns[i]], linewidth=0.3)
#     axs[i].set_title("10 min - " + slovnik[df.columns[i][-2:]], fontsize=45)
# fig.savefig("/home/kriza/programing/clanok/data/10min.pdf", format='pdf')

In [3]:
df

,30m-item56,30m-item57,30m-item58,30m-item59,30m-item60,30m-item61,30m-item62,30m-item63,30m-item64,30m-item65,...,30m-item68,30m-item69,30m-item70,30m-item71,30m-item72,30m-item73,30m-item74,30m-item75,30m-item76,30m-item77
0,9,10,10,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1,13,14,14,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2,22,22,22,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,26,26,26,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20,20,20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17468,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
17469,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
17470,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
17471,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [5]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
dataset = dataset.repeat()
list(dataset.as_numpy_iterator())

KeyboardInterrupt: 

In [37]:
def find_min_max(data):
    min = 99999
    max = -2
    for i in data:
        if i > max:
            max = i
        if i < min:
            min = i
    return min, max

# def normalize(data):

    


In [36]:
dataset = df["30m-item68"].values
dataset = dataset.astype("float32")
# normalize the dataset
X_train = dataset[:15552]
X_valid = dataset[15600:17460]

print(type(X_train))

<class 'numpy.ndarray'>


In [38]:
find_min_max(X_train)

(0.0, 1150.0)

In [ ]:

# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size : len(dataset), :]
# reshape into X=t and Y=t+1
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
trainX = trainX.reshape((3553, 10, 1))
testX = testX.reshape(((1745, 10, 1)))
print(testX.shape)
print(trainX.shape)
print(trainY.shape)

In [ ]:
type(trainX)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.LSTM(
  units=128,
  input_shape=(trainX.shape[1], trainX.shape[2]),
    return_sequences=True,
))
model.add(keras.layers.LSTM(
  units=64
))
model.add(keras.layers.Dense(units=1))
model.compile(
  loss='mean_squared_error',
  optimizer=keras.optimizers.Adam(0.0005)
)
model.summary()


In [ ]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(
    trainX, trainY,
    epochs=5,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=False,
    callbacks=[WandbCallback()]
)

model.save(os.path.join(wandb.run.dir, "model1.h5"))

In [ ]:
x_pred = model.predict(trainX)
y_pred = model.predict(testX)
print(len(x_pred[:,0]))
print(len(y_pred[:,0]))

In [ ]:
trainPredict = scaler.inverse_transform(x_pred)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(y_pred)
testY = scaler.inverse_transform([testY])
print(trainY)
print(testY)

In [ ]:
time = np.arange(0, len(dataset))

dataset1 = scaler.inverse_transform(dataset)
merged_list = []

for l in dataset1:
    merged_list.append(int(l[0]))
    
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=time, y=merged_list, mode="lines", name="povodne data",))
fig.add_trace(
    go.Scatter(
        x=time[look_back : len(trainPredict) + look_back],
        y=np.reshape(trainPredict, len(trainPredict)),
        mode="lines",
        name="treningova mnozina",
    )
)
fig.add_trace(
    go.Scatter(
        x=time[len(trainPredict) + (look_back * 2) + 1 : len(dataset) - 1],
        y=np.reshape(testPredict, len(testPredict)),
        mode="lines",
        name="testovacia mnozina",
    )
)
fig.show()